In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import joblib

import matplotlib.pyplot as plt
from sklearn import preprocessing
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import lightgbm as lgb
from lightgbm import LGBMRegressor
import holidays
us_holidays = holidays.country_holidays('US')
from sklearn.model_selection import RandomizedSearchCV

# Training loop
import torch
import torch.nn as nn
import torch.optim as optim

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [4]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [5]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:

execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# Weather Model

In [7]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "total_demand_1h",
            "demand_degrees_1h",
]

## XGBoost

In [8]:
model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 4.061524407266129
RMSE: 2.015322407771553
MAE: 1.067188920933303
MAPE: 0.4350008358415208
### Standard Deviation
MSE: 9.362222582871203e-16
RMSE: 0.0
MAE: 0.0
MAPE: 5.851389114294502e-17
### Variance
MSE: 8.765121169122354e-31
RMSE: 0.0
MAE: 0.0
MAPE: 3.4238754566884194e-33

Non-zero
MSE: 8.207877787744946
RMSE: 2.864939403852191
MAE: 1.7725544429383784
MAPE: 0.36717203592849273
### Standard Deviation
MSE: 0.0
RMSE: 0.0
MAE: 0.0
MAPE: 0.0
### Variance
MSE: 0.0
RMSE: 0.0
MAE: 0.0
MAPE: 0.0

#####
Zeros
MSE: 0.6713349950935068
RMSE: 0.8193503494192864
MAE: 0.4904598038079627
MAPE: 0.4904598038079627
### Standard Deviation
MSE: 1.1702778228589004e-16
RMSE: 0.0
MAE: 0.0
MAPE: 0.0
### Variance
MSE: 1.3695501826753678e-32
RMSE: 0.0
MAE: 0.0
MAPE: 0.0


In [ ]:
# overall
# MSE: 4.061524407266129
# RMSE: 2.015322407771553
# MAE: 1.067188920933303
# MAPE: 0.4350008358415209

# Non-zero
# MSE: 8.207877787744946
# RMSE: 2.864939403852191
# MAE: 1.7725544429383784
# MAPE: 0.36717203592849273

# Zeros
# MSE: 0.6713349950935068
# RMSE: 0.8193503494192864
# MAE: 0.4904598038079627
# MAPE: 0.4904598038079627

In [ ]:
#11-01
3.6281681076028174
1.9047750805811214
0.40089220936772335

#11-09 scaled
4.061524407266127
2.0153224077715524
0.4350008358415209

#11-09 unscaled
4.082018738616581
2.020400638144965
0.43368642099899685

#11-28
4.105374937296801
2.026172484586838
0.44111649375985346

In [ ]:
suf = "_b_xgb_01_27"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

## LightGBM

In [9]:
model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.094143 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1372
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 19
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093019 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1372
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 19
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

In [ ]:
# 11_15
4.659545812679481
2.158598112822181
0.531887442896664

#11-28
4.679057758183841
2.1631129785990932
0.5337148671777087

In [ ]:
suf = "_b_lgbm_01_27"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

## Random Forest

In [ ]:
model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='rf')

In [ ]:
4.830006668125328
2.1977276146341085
0.34375553207388226

#11-01
3.332732528910566
1.8255773138682914
0.37106450841048855

#11-28
3.761551942397637
1.9394720782722388
0.41353221812323404

0.41353221812323404

In [ ]:
suf = "_b_rf_01_27"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

filename = f'{models_dir}/test_predictions_list{suf}.sav'
joblib.dump(preds_list, filename)
del preds_list

## LSTM

In [ ]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "total_demand_1h",
            "demand_degrees_1h",
]

model, model_preds = train_model(df_train, df_test, FEATURES, model='lstm', epochs=20, batch_size=32)

Epoch [1/20], Loss: 0.0005
Epoch [2/20], Loss: 0.0002
Epoch [3/20], Loss: 0.0002
Epoch [4/20], Loss: 0.0001
Epoch [5/20], Loss: 0.0001
Epoch [6/20], Loss: 0.0001
Epoch [7/20], Loss: 0.0046
Epoch [8/20], Loss: 0.0002
Epoch [9/20], Loss: 0.0001
Epoch [10/20], Loss: 0.0001
Epoch [11/20], Loss: 0.0001
Epoch [12/20], Loss: 0.0001
Epoch [13/20], Loss: 0.0001
Epoch [14/20], Loss: 0.0001
Epoch [15/20], Loss: 0.0001
Epoch [16/20], Loss: 0.0001
Epoch [17/20], Loss: 0.0001
Epoch [18/20], Loss: 0.0001
Epoch [19/20], Loss: 0.0001
Epoch [20/20], Loss: 0.0001
19.632432310533225
4.430850066356706
1.0868993281445987


In [ ]:
suf = "_b_lstm_11_28"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model
